In [67]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly
import os
import time
import pickle
from datetime import datetime

# Global Model

In [128]:
no_of_noaa_records = 7431587

db = MongoClient(os.environ["DB_HOST"], int(os.environ["DB_PORT"]))
collection = 'noaa_nam_2'
pipeline = [
    {
        "$sample": {"size": 74315}
    }
]
cursor = db.sustaindb[collection].aggregate(pipeline)
df_original = pd.DataFrame(list(cursor))

In [129]:
def format_date(t):
    t = str(t)
    date = datetime.strptime(t, '%Y%m%d%H')
    return f'{date.year}-{date.month}-{date.day} {str(date.hour).zfill(2)}:{str(date.minute).zfill(2)}:{str(date.second).zfill(2)}'


df = df_original
df['year_month_day_hour'] = df['year_month_day_hour'].apply(format_date)
df

,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,60909b5dccef3c16a9f7d5e4,2010-1-14 12:00:00,0,G3900930,41.257586,-82.042522,102296.0,99069.0,253.225159,264.729492,...,4.676483,0.000,0.0,270.723434,20360.507202,24035.177785,5.953953,0.0,0.18736,0.0
1,60909bd2ba60520913262824,2010-1-15 12:00:00,3,G0800870,40.017721,-104.047304,102992.0,85858.0,1465.475159,270.553894,...,0.041497,0.000,0.0,271.623764,26067.372131,24035.608697,6.470860,0.0,0.00000,0.0
2,60909cdbf427f4b2b209d958,2010-1-18 00:00:00,0,G5600290,44.692840,-109.566195,101073.0,73462.0,2601.225159,261.439102,...,1.477570,0.000,0.0,271.571350,22160.520935,24234.948206,3.880981,0.0,1.27408,0.0
3,60909ae8e71b89653d2bde75,2010-1-13 06:00:00,6,G4100370,43.500737,-120.665510,100864.0,84813.0,1425.475159,274.918381,...,5.733231,4.250,0.0,274.628525,28167.625427,11035.114831,11.989980,100.0,0.00000,0.0
4,609099f30394db46bd12c006,2010-1-11 00:00:00,6,G0600070,39.408113,-121.597511,102178.0,101585.0,46.475159,280.707962,...,-2.079071,0.000,0.0,282.190552,20360.479736,24227.428675,19.429266,100.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74310,60909b82e7a50b8b8c8e62d2,2010-1-14 18:00:00,3,G2900810,40.244754,-93.816921,101905.0,98463.0,275.975159,273.235611,...,-2.752478,0.000,0.0,271.556381,26160.510254,37.607694,9.209002,100.0,0.10920,0.0
74311,60909cf1ef92c8286ad82459,2010-1-18 00:00:00,6,G2700710,47.883399,-94.019697,101581.0,96386.0,413.975159,262.300308,...,-1.697479,0.000,0.0,269.886429,30567.326355,24234.418267,4.614982,0.0,0.13280,0.0
74312,609099fe8ef9fc08588fcbbe,2010-1-11 06:00:00,0,G2700110,45.548602,-96.526505,102731.0,98336.0,337.225159,263.964081,...,-4.535324,0.000,0.0,264.617569,22160.514832,24234.747136,8.469556,100.0,0.13808,0.0
74313,609099baf93f3f2c0fafd78c,2010-1-10 12:00:00,6,G0400050,35.512395,-111.255848,103090.0,85958.0,1484.975159,278.918365,...,-0.220123,0.000,0.0,271.675156,20360.482788,24231.413460,6.015907,100.0,0.00000,0.0


In [131]:
noaa_features = [
    "mean_sea_level_pressure_pascal",
    "surface_pressure_surface_level_pascal",
    "orography_surface_level_meters",
    "temp_surface_level_kelvin",
    "2_metre_temp_kelvin",
    "2_metre_dewpoint_temp_kelvin",
    "relative_humidity_percent",
    "10_metre_u_wind_component_meters_per_second",
    "10_metre_v_wind_component_meters_per_second",
    "total_precipitation_kg_per_squared_meter",
    "water_convection_precipitation_kg_per_squared_meter",
    "soil_temperature_kelvin",
    "pressure_pascal",
    "visibility_meters",
    "precipitable_water_kg_per_squared_meter",
    "total_cloud_cover_percent",
    "snow_depth_meters",
    "ice_cover_binary"
]

unique_timestamps = df['year_month_day_hour'].unique()
df_map = {}

for selected_feature in noaa_features:
    df_s = df[['year_month_day_hour', selected_feature]]
    means = []
    for t in unique_timestamps:
        mean = df_s[df_s['year_month_day_hour'] == t][selected_feature].mean()
        means.append(mean)
    df_means = pd.DataFrame(list(zip(unique_timestamps, means)), columns=['ds', 'y'])
#     df_means.index.name = 'ds'
    df_map[selected_feature] = df_means
    
df_map['mean_sea_level_pressure_pascal'].head()

,ds,y
0,2010-1-14 12:00:00,102212.554850
1,2010-1-15 12:00:00,102427.085850
2,2010-1-18 00:00:00,101182.639580
3,2010-1-13 06:00:00,102174.151277
4,2010-1-11 00:00:00,102747.603226


In [116]:
pickle.dump(df_map, open('pickles/noaa/df_map.pkl', 'wb'))

In [117]:
# Start by loading pickled 'df_map'
df_map = pickle.load(open('pickles/noaa/df_map.pkl', 'rb'))

In [132]:
# select one feature
df0 = df_map['mean_sea_level_pressure_pascal']
df0.head()
df0.columns

Index(['ds', 'y'], dtype='object')

## Build Global Model

In [134]:
def predict(df_train):
    m = Prophet(
        daily_seasonality=True,
        weekly_seasonality=True,
        yearly_seasonality=True,
    )
    # model.fit(df, algorithm='LBFGS')
    fit_m = m.fit(df_train, algorithm='LBFGS')
    df_train_future = m.make_future_dataframe(periods=300, freq='H')
    df_train_forecast = m.predict(df_train_future)

    return fit_m, df_train_future, df_train_forecast

fit_m, df_train, df_train_forecast = predict(df0)

Initial log joint probability = -2.02015
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       340.026    0.00119618       6205.51           1           1      127   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       347.638   6.00432e-05       2010.85           1           1      240   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       349.891    0.00194563       14165.1           1           1      346   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       352.741   0.000294412       4239.46           1           1      453   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499        353.66   4.30424e-05       1226.97           1           1      564   
    Iter      log prob        ||dx||      ||grad||       alpha  

In [138]:
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df0_cv = cross_validation(fit_m, initial='2 days', period='1 days', horizon='1 days')
df0_p = performance_metrics(df0_cv)
df0_p.head()

INFO:prophet:Making 8 forecasts with cutoffs between 2010-01-12 18:00:00 and 2010-01-19 18:00:00


  0%|          | 0/8 [00:00<?, ?it/s]

INFO:prophet:n_changepoints greater than number of observations. Using 8.


Initial log joint probability = -2.02
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       99.5234     0.0133659         10367      0.4166      0.4166      132   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199        109.86    0.00447523       34808.5      0.9696      0.9696      247   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       110.879   0.000324645       2990.69           1           1      359   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       112.108   0.000734205       2458.97           1           1      470   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       112.265   6.11675e-06       576.431           1           1      580   
    Iter      log prob        ||dx||      ||grad||       alpha     

INFO:prophet:n_changepoints greater than number of observations. Using 11.


...

Initial log joint probability = -2.02007
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       247.446     0.0299858       32558.6           1           1      138   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       251.443    0.00921149       20905.1           1           1      256   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       253.238    0.00027131       4269.97           1           1      371   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       254.584    0.00136367       5441.74           1           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499        255.97   0.000254056       3333.44           1           1      585   
    Iter      log prob        ||dx||      ||grad||       alpha  

INFO:prophet:n_changepoints greater than number of observations. Using 24.


Initial log joint probability = -2.02016
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       271.872   0.000514403       5725.71           1           1      133   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       273.314   0.000242394       7213.43      0.1967      0.8346      244   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       280.163   0.000136406       8901.12       1.354      0.1354      353   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       283.142   0.000156606       2157.45           1           1      464   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       283.561   0.000140964        3242.1           1           1      579   
    Iter      log prob        ||dx||      ||grad||       alpha  

Initial log joint probability = -2.02013
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       312.062    0.00604464       7972.53           1           1      131   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       321.024   0.000525731       2394.94      0.3284           1      239   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       328.209    7.3618e-05       1384.28       1.316      0.1316      355   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       331.488    3.1282e-05       899.346           1           1      467   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       332.364   0.000332871       1341.81      0.2982           1      584   
    Iter      log prob        ||dx||      ||grad||       alpha  

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,0 days 06:00:00,8551.897334,92.476469,83.105487,0.000817,0.000699,0.000817,0.000
1,0 days 12:00:00,36077.301491,189.940258,165.216841,0.001624,0.001437,0.001624,0.125
2,0 days 18:00:00,125281.745993,353.951615,293.815574,0.002887,0.002345,0.002889,0.250
3,1 days 00:00:00,299149.801304,546.945885,418.104264,0.004113,0.003543,0.004121,0.125


In [139]:
model = fit_m
print(f'seasonality_prior_scale: {model.seasonality_prior_scale}')
print(f'changepoint_prior_scale: {model.changepoint_prior_scale}')
print(f'uncertainty_samples: {model.uncertainty_samples}')
print(f'seasonlity_mode: {model.seasonality_mode}')
print(f'interval_width: {model.interval_width}')
print(f'growth: {model.growth}')

seasonality_prior_scale: 10.0
changepoint_prior_scale: 0.05
uncertainty_samples: 1000
seasonlity_mode: additive
interval_width: 0.8
growth: linear


## GridSearch on Global Model

# Child Models (Non-TL)

In [10]:
cursor = db.sustaindb[collection].aggregate([{"$match": {"gis_join": "G1600810"}}])
df = pd.DataFrame(list(cursor))
df

,_id,year_month_day_hour,timestep,gis_join,latitude,longitude,mean_sea_level_pressure_pascal,surface_pressure_surface_level_pascal,orography_surface_level_meters,temp_surface_level_kelvin,...,10_metre_v_wind_component_meters_per_second,total_precipitation_kg_per_squared_meter,water_convection_precipitation_kg_per_squared_meter,soil_temperature_kelvin,pressure_pascal,visibility_meters,precipitable_water_kg_per_squared_meter,total_cloud_cover_percent,snow_depth_meters,ice_cover_binary
0,60909976bf51472ccaadb923,2010011000,0,G1600810,43.629500,-111.135989,103213.0,79356.0,2120.975159,254.399887,...,0.508087,0.000,0.0,272.680527,18560.493469,24037.076426,5.508711,0.0,1.13516,0.0
1,60909976bf51472ccaadb924,2010011000,0,G1600810,43.719985,-111.295273,103234.0,80422.0,2016.225159,257.774887,...,0.508087,0.000,0.0,266.680527,18560.493469,24037.076426,5.633711,0.0,0.17268,0.0
2,60909976bf51472ccaadb925,2010011000,0,G1600810,43.732348,-111.153005,103242.0,79986.0,2058.975159,254.899887,...,0.383087,0.000,0.0,266.680527,18560.493469,24037.076426,5.508711,0.0,0.17268,0.0
3,60909976bf51472ccaadb926,2010011000,0,G1600810,43.822758,-111.312472,103255.0,81177.0,1937.725159,257.774887,...,0.383087,0.000,0.0,267.680527,18560.493469,24037.076426,5.758711,0.0,0.17316,0.0
4,60909976bf51472ccaadb927,2010011000,0,G1600810,43.925461,-111.329707,103344.0,82978.0,1766.475159,257.774887,...,0.133087,0.000,0.0,267.680527,18560.493469,24037.076426,5.883711,0.0,0.17264,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
781,60909f64e204dece8a2fde33,2010012018,3,G1600810,43.719985,-111.295273,99931.0,78062.0,2016.225159,273.282944,...,1.319046,0.000,0.0,269.487732,26060.475159,24037.724245,5.895195,92.0,0.17048,0.0
782,60909f64e204dece8a2fde34,2010012018,3,G1600810,43.732348,-111.153005,99924.0,77659.0,2058.975159,273.282944,...,1.444046,0.000,0.0,269.487732,27460.475159,24037.724245,5.645195,86.0,0.17048,0.0
783,60909f64e204dece8a2fde35,2010012018,3,G1600810,43.822758,-111.312472,99976.0,78821.0,1937.725159,273.407944,...,0.069046,0.000,0.0,269.487732,27860.475159,24037.724245,6.145195,72.0,0.15600,0.0
784,60909f64e204dece8a2fde36,2010012018,3,G1600810,43.835125,-111.170056,99966.0,78821.0,1936.225159,273.282944,...,0.569046,0.000,0.0,269.487732,28060.475159,24037.724245,5.895195,46.0,0.15984,0.0


# Child Models (TL)